In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import dlib
import cv2
import glob as gb
from facenet_pytorch import MTCNN, extract_face

os.sys.path.append('../')

from PIL import Image
from utils .align import AlignDlib

# To compile dlib
# sudo apt install -y cmake
# pip install dlib

In [2]:
root_data = '/media/angelo/DATEN/Datasets/ICB-RW/'

# Recent fix to identify folders with organized data
list_of_people = [str(i) for i in range(1,189)]

with open(root_data + 'failures.txt') as f:
    failures = [line.rstrip('\n') for line in f]
    
list_of_people = [x for x in list_of_people if x not in failures]

In [ ]:
# Let's process every probe and watch-list picture and create a folder for their faces

mtcnn = MTCNN(keep_all=True)

for i, subject in enumerate(list_of_people):
    
    face_id = '00' + subject + '_'
    gallery_faces_path = root_data + 'gallery/'
    probe_faces_path = root_data + 'probe/'
    probe_counter = 1
    
    if not os.path.exists(gallery_faces_path):
        os.mkdir(gallery_faces_path) 
    
    if not os.path.exists(probe_faces_path):
        os.mkdir(probe_faces_path) 
    
    for image in os.listdir(probe_faces_path + subject):
        
        image_name = os.path.splitext(image)[0]
        image_path = probe_faces_path + subject + '/' + image
        image_RGB = Image.open(image_path)
        
        faces, _ = mtcnn.detect(image_RGB)
        isEmpty = (str(type(faces)) == "<class 'NoneType'>")

        if isEmpty: # Checking if faces were detected
            print('Subject ' + face_id + ' not complete.')

        if not isEmpty: 
            if image_name in ['f','F']:
                extract_face(image_RGB, faces[0], image_resize=False,
                             save_path=gallery_faces_path + face_id + str(image_name).lower() + '.jpg')
            else:
                extract_face(image_RGB, faces[0], image_resize=False,
                             save_path=probe_faces_path + subject + '/' + face_id + str(probe_counter) + '.jpg')
                probe_counter+=1

    print('Processed subject {}/{}'.format(i+1,len(list_of_people)))

Processed subject 1/90
Processed subject 2/90
Processed subject 3/90
Processed subject 4/90
Processed subject 5/90
Processed subject 6/90
Processed subject 7/90
Processed subject 8/90
Processed subject 9/90
Processed subject 10/90


In [ ]:
# Alternative

# Let's process every probe and watch-list picture and create a folder for their faces

# DLIB detection
detector = dlib.get_frontal_face_detector()

for i, subject in enumerate(list_of_people):
    
    face_id = '00' + subject + '_'
    gallery_faces_path = root_data + 'gallery/'
    probe_faces_path = root_data + 'probe/'
    probe_counter = 0 # For naming according to ICB_RW official data 
    
    if not os.path.exists(gallery_faces_path):
        os.mkdir(gallery_faces_path) 
    
    if not os.path.exists(probe_faces_path):
        os.mkdir(probe_faces_path) 
    
    for image in os.listdir(root_data + subject):
        
        image_name = os.path.splitext(image)[0]
        image_RGB = dlib.load_rgb_image(root_data + subject + '/' + image)
        faces = detector(image_RGB, 1)
        for face in faces: # We have only one face per image
            if image_name in ['f','r','l']:
                cv2.imwrite(gallery_faces_path + face_id + image_name + '.jpg', 
                            image_RGB[face.top():face.bottom(), 
                                   face.left():face.right()][...,::-1])
            else:
                probe_counter += 1
                cv2.imwrite(probe_faces_path + face_id + str(probe_counter) + '.jpg', 
                            image_RGB[face.top():face.bottom(), 
                                   face.left():face.right()][...,::-1])
    break